In [2]:
import pandas as pd
df=pd.read_csv('afterPP_dataset.csv')

In [3]:
df

,Reviews
0,كان لي تجربه اكثر من راءعه اليوم في هذا الفرع ...
1,اسوء خدمه شركه اتصالات علي المستوي العربي كله ...
2,اشكر الموظفه ضيء علي تعاملها ورقيها وخدمتها لي...
3,اسوا نت في العالم عانيت مليون مرررررا منهم رفع...
4,الفرع جدا ممتاز واشكر الاخوات الموظفات علي رحا...
...,...
382,التذكره ايش صار عليها محد فيكم تكرم تواصل معي ...
383,قويه
384,انترنت لا محدود سرعه ميقا زين السعوديه
385,سرعه عاليه ميجا ال زين يارب وماخاب من قال يارب


In [4]:
df.dropna(inplace=True)

In [6]:
df_train= pd.read_table('ar_reviews_100k.tsv')
df_train

,label,text
0,Positive,ممتاز نوعا ما . النظافة والموقع والتجهيز والشا...
1,Positive,أحد أسباب نجاح الإمارات أن كل شخص في هذه الدول...
2,Positive,هادفة .. وقوية. تنقلك من صخب شوارع القاهرة الى...
3,Positive,خلصنا .. مبدئيا اللي مستني ابهار زي الفيل الاز...
4,Positive,ياسات جلوريا جزء لا يتجزأ من دبي . فندق متكامل...
...,...,...
99994,Negative,معرفش ليه كنت عاوزة أكملها وهي مش عاجباني من ا...
99995,Negative,لا يستحق ان يكون في بوكنق لانه سيئ . لا شي. لا...
99996,Negative,كتاب ضعيف جدا ولم استمتع به. فى كل قصه سرد لحا...
99997,Negative,مملة جدا. محمد حسن علوان فنان بالكلمات، والوصف...


In [8]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import make_pipeline
from nltk.corpus import stopwords
import nltk

# Download stopwords from NLTK
nltk.download('stopwords')

# Load Arabic stopwords
arabic_stopwords = stopwords.words('arabic')

#split the data
X_train, X_test, y_train, y_test = train_test_split(df_train['text'], df_train['label'], test_size=0.33, random_state=42)

# Create a TF-IDF Vectorizer and Logistic Regression pipeline
model = make_pipeline(
    TfidfVectorizer(stop_words=arabic_stopwords),
    LogisticRegression()
)

# Train the model on your labeled data
model.fit(X_train, y_train)

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Admin\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
c:\Users\Admin\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\feature_extraction\text.py:408: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['آمين', 'أب', 'أخ', 'أفعل', 'أفعله', 'ؤلاء', 'إل', 'إم', 'ات', 'اتان', 'ارتد', 'ان', 'انفك', 'برح', 'تان', 'تبد', 'تحو', 'تعل', 'حد', 'حم', 'حي', 'خب', 'ذار', 'سيما', 'صه', 'ظل', 'ظن', 'عد', 'قط', 'مر', 'مكان', 'مكانكن', 'نب', 'هات', 'هب', 'واها', 'وراء'] not in stop_words.
  warnings.warn(
c:\Users\Admin\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\linear_model\_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.o

Pipeline(steps=[('tfidfvectorizer',
                 TfidfVectorizer(stop_words=['إذ', 'إذا', 'إذما', 'إذن', 'أف',
                                             'أقل', 'أكثر', 'ألا', 'إلا',
                                             'التي', 'الذي', 'الذين', 'اللاتي',
                                             'اللائي', 'اللتان', 'اللتيا',
                                             'اللتين', 'اللذان', 'اللذين',
                                             'اللواتي', 'إلى', 'إليك', 'إليكم',
                                             'إليكما', 'إليكن', 'أم', 'أما',
                                             'أما', 'إما', 'أن', ...])),
                ('logisticregression', LogisticRegression())])

In [9]:
#test the model through classification report
from sklearn.metrics import classification_report
y_pred = model.predict(X_test)
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

       Mixed       0.59      0.55      0.57     11105
    Negative       0.71      0.72      0.72     10962
    Positive       0.68      0.72      0.70     10933

    accuracy                           0.66     33000
   macro avg       0.66      0.66      0.66     33000
weighted avg       0.66      0.66      0.66     33000



In [10]:
#hyperparameter tuning
from sklearn.model_selection import GridSearchCV
param_grid = {'logisticregression__C': [0.001, 0.01, 0.1, 1, 10]}
grid = GridSearchCV(model, param_grid, cv=5, verbose=0)
grid.fit(X_train, y_train)
print("Best cross-validation score: {:.2f}".format(grid.best_score_))
print("Best parameters: ", grid.best_params_)
print("Test-set score: {:.2f}".format(grid.score(X_test, y_test)))

c:\Users\Admin\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\feature_extraction\text.py:408: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['آمين', 'أب', 'أخ', 'أفعل', 'أفعله', 'ؤلاء', 'إل', 'إم', 'ات', 'اتان', 'ارتد', 'ان', 'انفك', 'برح', 'تان', 'تبد', 'تحو', 'تعل', 'حد', 'حم', 'حي', 'خب', 'ذار', 'سيما', 'صه', 'ظل', 'ظن', 'عد', 'قط', 'مر', 'مكان', 'مكانكن', 'نب', 'هات', 'هب', 'واها', 'وراء'] not in stop_words.
  warnings.warn(
c:\Users\Admin\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\feature_extraction\text.py:408: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['آمين', 'أب', 'أخ', 'أفعل', 'أفعله', 'ؤلاء', 'إل', 'إم', 'ات', 'اتان', 'ارتد', 'ان', 'انفك', 'برح', 'تان', 'تبد', 'تحو', 'تعل', 'حد', 'حم', 'حي', 'خب', 'ذار', 'سيما', 'صه', 'ظل', 'ظن', 'عد', 'قط', 'مر', 'مكان', 'مكانكن', 'نب', 'هات', 'هب', 'واها'

KeyboardInterrupt: 

In [ ]:
# Predict labels for the unlabeled dataset
predicted_labels = model.predict(df['text'])

# Add the predicted labels to your unlabeled dataset
df['predicted_label'] = predicted_labels

# Now df contains an additional column 'predicted_label' with the predicted labels